<a href="https://colab.research.google.com/github/MoSaeedd/BinaryClassifier/blob/main/BinaryClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch_lightning
import pytorch_lightning as pl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 7.3 MB/s eta 0:00:00


In [ ]:
import torch
torch.__version__
from torchvision.models import resnet18
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from pandas.core.arrays import boolean
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
import torchvision.datasets as datasets
from  torchvision import transforms
import numpy as np

In [ ]:
dataset = datasets.MNIST(
    root='./data',
    transform=transforms.ToTensor(),download = True
)

# Just removing the all classes but 0,1
targets = dataset.targets
target_indices = np.arange(len(targets))
train_idx, val_idx = train_test_split(target_indices, train_size=0.8)
classidx_to_remove = 0
idx_to_keep0Tr = targets[train_idx]==0
idx_to_keep1Tr = targets[train_idx]==1

idx_to_keep0Test = targets[val_idx]==1
idx_to_keep1Test = targets[val_idx]==0
# keeping needed classes
train_idx = train_idx[np.logical_or(np.array(idx_to_keep0Tr, dtype=bool), 
                                   np.array(idx_to_keep1Tr, dtype=bool))]
val_idx = val_idx[np.logical_or(np.array(idx_to_keep0Test, dtype=bool),
                                np.array(idx_to_keep1Test, dtype=bool))]

train_ds = Subset(dataset, train_idx)
test_ds = Subset(dataset, val_idx)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
print(max(train_ds.dataset.targets[train_ds.indices]))
print(sum(train_ds.dataset.targets[train_ds.indices]==1))
print(sum(train_ds.dataset.targets[train_ds.indices]==0))
print(sum(train_ds.dataset.targets[test_ds.indices]==1))
print(sum(train_ds.dataset.targets[test_ds.indices]==0))


tensor(1)
tensor(5431)
tensor(4739)
tensor(1311)
tensor(1184)


In [ ]:
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)

test_dl = DataLoader(test_ds, batch_size=64)

In [ ]:
class ResNetMNIST(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = resnet18(num_classes=1)
    self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    self.loss = nn.BCEWithLogitsLoss()


  def forward(self, x):
    return self.model(x)
  
  def training_step(self, batch, batch_no):
    x, y = batch
    logits = self(x)
    loss = self.loss(logits, y.unsqueeze(1).float())
    return loss
  
  def configure_optimizers(self):
    return torch.optim.RMSprop(self.parameters(), lr=0.005)

In [ ]:
model = ResNetMNIST()

In [ ]:
trainer = pl.Trainer(
    gpus=1,
    max_epochs=6)

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, train_dl)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type              | Params
--------------------------------------------
0 | model | ResNet            | 11.2 M
1 | loss  | BCEWithLogitsLoss | 0     
--------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.683    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=6` reached.


In [ ]:
trainer.save_checkpoint("resnet18_BinaryMnist.pt")


In [ ]:
def get_prediction(x, model: pl.LightningModule):
  model.freeze() # prepares model for predicting
  pred = torch.sigmoid(model(x))
  prob = [(1-p1,p1) for p1 in pred ]
  probabilities = torch.tensor(prob)
  predicted_class = torch.tensor([1 if p1>0.5 else 0 for p1 in pred ])
  # print(max(predicted_class))
  return predicted_class, probabilities

In [ ]:
from tqdm.autonotebook import tqdm

In [ ]:
inference_model = ResNetMNIST.load_from_checkpoint("resnet18_BinaryMnist.pt", map_location="cuda")

In [ ]:
true_y, pred_y = [], []
for batch in tqdm(iter(test_dl), total=len(test_dl)):
  x, y = batch
  true_y.extend(y)
  preds, probs = get_prediction(x, inference_model)
  pred_y.extend(preds.cpu())

  0%|          | 0/39 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_y, pred_y, digits=3))

              precision    recall  f1-score   support

           0      0.999     0.999     0.999      1184
           1      0.999     0.999     0.999      1311

    accuracy                          0.999      2495
   macro avg      0.999     0.999     0.999      2495
weighted avg      0.999     0.999     0.999      2495

